# Single-Qubit Geometric Algebra

## Preliminaries

The first order of business is to import the necessary modules.  The following code requires a local installation of the "clifford" python package.  In anaconda, this can be accomplished using the following command.
    
    conda install -c conda-forge clifford
    
The presents of module QuantumGA.py in the same directory as this notebook or a directory on the PYTHONPATH is also required.

In [38]:
from clifford import MVArray
from QuantumGA import _e1, _e2, _e3, _e12, _e13, _e23, _e123, _zero, _one 
from QuantumGA import tract, mvec, places, linelen, comm
from numpy import sqrt, e, pi, array

The next two commands affect the display formatting and can be changed at any time to suit the user's preferences.

In [39]:
places(8)
linelen(120)

Now define the primitive blades that we will be using.

In [40]:
e1 = _e1
e2 = _e2
e3 = _e3

e12 = _e12
e23 = _e23
e13 = _e13

e123 = _e123

i = e123
zero = _zero
one = _one

## Primitive spin states and the specular matrix

Define the spin-up, spin-not-up, spin-down and spin-not-down states (in that order).  Note: further information about our naming convections can be found in the document SingleQubitGA.pdf.

In [41]:
u_ket = (1+e3)/2
n_ket = (1-e3)/2
d_ket = (e1+e13)/2
p_ket = (e1-e13)/2

In the Pauli and Dirac algebras hermitian conjugation is used to transform a column-vector state (ket) into a row-vector state (bra).  In quantum geometric algebra the equivalent operation is *reversion* which is implemented in the "clifford" library with the unary operator tilda. 

In [42]:
~d_ket

0.5^e1 - 0.5^e13

Dirac inner and outer products, quantum expectation values and probabilities can all be calculated in the bra-ket formalism using just multivectors and the reversion operator.  For example, the Dirac inner product of the spin-down state ,$\langle d| d\rangle$, can be easily calculated.

In [43]:
~d_ket*d_ket

0.5 + 0.5^e3

The Dirac inner product is usually considered a complex function.  However, in quantum geometric algebra the result is a complex number times the base state, in this case *u_ket*.  To isolate just the complex factor from the base state we use the built-in *ufact* method.  This function finds a multivector that can be right multiplied by the spin-up state ,$|u\rangle$, to obtain the same result.

In [44]:
(~d_ket*d_ket).ufact()

1.0

We are now ready to define the 2x2 specular matrix.  The syntax here is unfortunately complicated by a need to use the MVArray method provided by the clifford package.

In [45]:
S2 = array(MVArray([u_ket, p_ket, d_ket, n_ket]).reshape(2,2))

The resulting object is a 2x2 numpy array with multivector elements. In the following sections we will be using this array along with the *tract* method to create a series of multivector objects. 

In [46]:
S2

array([[0.5 + 0.5^e3, 0.5^e1 - 0.5^e13],
       [0.5^e1 + 0.5^e13, 0.5 - 0.5^e3]], dtype=object)

Notice that the two states that make up the left column of the 2x2 spectral matrix, *u_ket* and *d_ket*, define the basis vectors of the Pauli spin algebra, $|u\rangle$ and $|d\rangle$ (sometimes written $|0\rangle$ and $|1\rangle$).  These two states are often depicted as a column vector and referred to as a Pauli spinor.

In quantum geometric algebra these two states can be treated as two pieces of a single multivector located in a particular two-dimensional subspace of $\mathbb{G}_{3}$, *span*{$1, \mathbf{e}1, \mathbf{e}3, \mathbf{e}13$}. 

## Making multivectors from matrices

To translate a typical 2x2 matrix into its multivector equivalent form we start with a, possibly complex, 2x2 numpy array.  For example the Pauli $\sigma_{1}$ matrix. 


In [47]:
pauli_x = array([[0,1], [1,0]])
pauli_x

array([[0, 1],
       [1, 0]])

Contracting this matrix with the 2x2 spectral matrix we can generate a mulivector which contains the same mathematical properties as the matrix form.

In [48]:
X = tract(pauli_x, S2)
X

1.0^e1

We can do the same for the other two Pauli matrices.  Note the use of complex elements in the case of the $\sigma_{2}$ (pauli_y) matrix.

In [49]:
pauli_y = array([[0, -1j], [1j, 0]])
pauli_z = array([[1,0], [0,-1]])

In [50]:
Y = tract(pauli_y, S2)
Y

1.0^e2

In [51]:
Z = tract(pauli_z, S2)
Z

1.0^e3

These simple calculations demonstrate that, within this context, the Pauli matrices and the basis vectors of $\mathbb{G}_{3}$ are one and the same.

To help convince readers that this is the case, we can examine the commutation relations between our multivectors vs. those of  the Pauli matrices $[\sigma_{a},\sigma_{b}]=2i\epsilon_{abc}\sigma_{c}$.  To do this we employ the built-in function *comm* which calculates the commutator of two multivectors.

In [52]:
comm(X, Y)

2.0^e12

This result is equivalent to the value $2i\sigma_{3}$ where the imaginary unit *i* has been replaced by the $\mathbb{G}_{3}$ pseudoscalar $\mathbf{e}123$.

In [53]:
2*i*Z

2.0^e12

The remaining combinations can be similarly calculated.

## Quantum operators

Building upon the previous results we next define some other well-known quantum states namely the right, left, in and out spin states. 

In [54]:
r_ket = (u_ket+d_ket)/sqrt(2)
l_ket = (u_ket-d_ket)/sqrt(2)

i_ket = (u_ket+i*d_ket)/sqrt(2)
o_ket = (u_ket-i*d_ket)/sqrt(2)

We can now calculate, for an example, the effect that a $\sigma_{3}$ matrix has on the spin-right state.

In [55]:
Z*r_ket

0.35355339 - 0.35355339^e1 + 0.35355339^e3 - 0.35355339^e13

If this result seems strange to you then consider the multivector form of the spin-left state.

In [56]:
l_ket

0.35355339 - 0.35355339^e1 + 0.35355339^e3 - 0.35355339^e13

You can see that, when the spin-right state is left-multiplied by the $\sigma_{3}$ matrix, the resulting state is spin-left.  It may be informative if we apply the *ufact* function to our result.

In [57]:
(Z*r_ket).ufact()

0.70710678 - 0.70710678^e13

Many readers will recognize this multivector as the rotor $e^{-e13*\pi/4}$ or alternately as a unit quaternion.  To emphasize this interpretation we apply the *dequat* function which decomposes an even multivector into its modulus and its argument.

In [58]:
(Z*r_ket).ufact().dequat()

(1.0, 0.7853981633974483, -1.0^e13)

The output is a tuple which contains the modulus, the argument and the unit bivector that define the transformation.  

The QuantumGA module allows us to produce this or any other rotor using exponentiation in a straightforward way. 

In [59]:
e**(-e13*pi/4)

0.70710678 - 0.70710678^e13

It is notable that this use of contraction with the specular matrix, in order to convert 2x2 matrices into their multivector equivalent form, can be used as a general tool.  We can easily apply it to other well-known quantum operators.  For example the Hadamard quantum gate.

In [61]:
hadamard = array([[1,1], [1,-1]])/sqrt(2)
H = tract(hadamard, S2)
H

0.70710678^e1 + 0.70710678^e3

The multivector form of a Hadamard gate is nothing more than the unit vector $(\mathbf{e}1+\mathbf{e}3)\ /\sqrt{2}$.   Its known effect on the spin-up state is to produce the spin-right state, which we can verify.

In [62]:
H*u_ket

0.35355339 + 0.35355339^e1 + 0.35355339^e3 + 0.35355339^e13

But because a Hadamard gate is a unit vector it can also be applied using the sandwich product to generate a reflection.

In [63]:
H*u_ket*H

0.5 + 0.5^e1

This result shows that the vector portion of *u\_ket* has been transformed from $\mathbf{e}3$ into $\mathbf{e}1$, its reflection in vector H.

This may help to demonstrate that the geometric nature of a Hadamard gate's action can be more easily appreciated once it has been translated into multivector form.  Any other single-qubit quantum gate can be easily translated into a multivector and scrutinized using this same methodology.

## Sandbox

Readers are encouraged at this point to play around with the various operators and states that we have defined so far.  Just enter a command in the following box and press run.  Verify for yourself that these multivectors behave in the same manner as the operators and states that you may be familiar with.

Python programmers can load QuantumGA.py and Definitions.py into any Python 3 environment and discover even more.     